In [15]:
from datasets import load_dataset_builder, get_dataset_split_names
from pprint import pprint

datasets_names = ["its5Q/yandex-q", 'bragovo/diasum']
for dataset_name in datasets_names:
    build = load_dataset_builder(dataset_name)
    # print split names, size (amount of samples) and features
    print(f"Dataset: {dataset_name}")
    pprint(build.info.splits)
    print(f"Features: {build.info.features}")
    print(f"Size: {build.info.splits.total_num_examples}")

Dataset: its5Q/yandex-q
{'train': SplitInfo(name='train',
                    num_bytes=1146588040,
                    num_examples=836810,
                    shard_lengths=[366000, 366000, 104810],
                    dataset_name='yandex-q')}
Features: {'description': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answer': Value(dtype='string', id=None)}
Size: 836810
Dataset: bragovo/diasum
{'test': SplitInfo(name='test',
                   num_bytes=42796,
                   num_examples=40,
                   shard_lengths=None,
                   dataset_name='json'),
 'train': SplitInfo(name='train',
                    num_bytes=396463,
                    num_examples=352,
                    shard_lengths=None,
                    dataset_name='json')}
Features: {'id': Value(dtype='string', id=None), 'summary': Value(dtype='string', id=None), 'dialogue': Value(dtype='string', id=None)}
Size: 392


In [17]:
from datasets import load_dataset, concatenate_datasets
yandex = load_dataset("its5Q/yandex-q", split="train")
diasum = load_dataset("bragovo/diasum")
# diasum - concatenation of train, test splits
diasum = concatenate_datasets([diasum["train"], diasum["test"]])

In [21]:
pprint(yandex[0])

{'answer': 'Никак но можно попробовать найти сайты которые за тебя пройдут '
           'капчу за тебя попробуй это а так не вариант ну или зайди с другого '
           'браузера если с пк и пробуй опять',
 'description': '',
 'question': 'Как войти в Роблокс, если не работает верификация, пишет слишком '
             'медленно?'}


In [22]:
pprint(diasum[0])

{'dialogue': 'Брэд: Эй, ты помнишь меня?\r\n'
             'Клаудия: Нет, не совсем.\r\n'
             'Клаудия: Я должна помнить?\r\n'
             'Брэд: Мы встретились на конференции в прошлом месяце.\r\n'
             'Брэд: Ты должна помнить меня.\r\n'
             'Клаудия: Подожди секунду...\r\n'
             'Клаудия: О, да, Брэд из утренней сессии.\r\n'
             'Клаудия: Должно быть, у тебя есть причина, почему ты пишешь.\r\n'
             'Брэд: Да, я понравился твоя презентация, но я забыл спросить '
             'тебя о источниках, которые ты использовал.\r\n'
             'Клаудия: Спасибо за любезные слова. Вот:\r\n'
             'Клаудия: <file_other>\r\n'
             'Брэд: Большое спасибо. Это очень много значит.\r\n'
             'Клаудия: Рада, что смогла помочь :)\r\n'
             'Брэд: До свидания :)',
 'id': '15037',
 'summary': 'Брэд и Клаудия встретились на конференции. Брэд понравилась '
            'презентация Клаудии и он хочет знать источники, котор

In [26]:
import numpy as np
# Looking on dialogues you can note that each replica is separated by '\n' and each speaker is separated by ': '
n = 1
for i, dialog in enumerate(np.random.choice(diasum, n)):
    print(dialog['dialogue'])

Эмма: Мы идем на пляж, хотите присоединиться?

Шароль: Конечно, кто еще придет?

Эмма: Это я, Анна, Эмили, Венди, Кейт и ты.

Шароль: Венди будет ли в порядке со мной, присоединившись?

Эмма: Я думаю, что да ... есть ли какая-то проблема между вами?

Шароль: Я думаю, что она меня не любит ... она всегда пытается избегать меня ...

Эмма: Действительно? Тогда я думаю, что вы должны обязательно присоединиться и разобраться с этим.

Шароль: Хмм .. я не хочу быть ее другом по принуждению ...

Эмма: Я знаю, что она вас не ненавидит, должно быть какое-то недоразумение ... давайте встретимся и разберемся. Будьте у меня в 11 утра.

Шароль: Конечно, я буду там.


In [108]:
def format_dialogue(history, speech1, speech2,  user1=1, user2=2):
    return f'history: {history}\n{user1}: {speech1}\t{user2}: {speech2}'

def proceed_dialogue(dialogue, sep=': '):
    # remove empty lines
    lines = list(filter(lambda x: x.strip(), dialogue.split('\n'))) 
    # split by first ': ' to get speaker and text. Lower and strip text
    splits = list(map(lambda x: x.lower().strip().split(sep, 1), lines)) 
    splits = list(filter(lambda x: len(x) == 2, splits)) # remove lines without ': '
    res = [] # list of dialogues
    history = '' # history of dialogues
    prev_speaker, prev_text = splits[0] # first speaker and text

    dialogue_dict = {prev_speaker: 1} # {speaker: number}
    last = 2 # next number
    for speaker, text in splits[1:]:
        speaker, text = speaker.strip(), text.strip()
        if speaker not in dialogue_dict:
            dialogue_dict[speaker] = last
            last += 1
        res.append(format_dialogue(history, prev_text, text, dialogue_dict[prev_speaker], dialogue_dict[speaker]))
        history += f'{dialogue_dict[prev_speaker]}: {prev_text}\t'
        prev_speaker, prev_text = speaker, text
    return res

# n = np.random.randint(0, len(diasum))
problem = diasum[306]
dialogue = problem['dialogue']
print(f'Before\n{dialogue}')
res = proceed_dialogue(dialogue)

print(f'After')
for i, line in enumerate(res):
    print(f'--------{i}--------')
    print(line)



Before
Джонни: Девушка в видео на вторник очень сексуальная.
Люк: <file_video>
Люк: На этот вторник?
Джонни: Она стоит того, чтобы нарушить правило "нет нута в ноябре".
Люк: Джонни, найди себе девушку. Влюбись в нее со всей душой и перестань говорить как подлец.
Джонни: Человек, это же шутка. Я уважаю женщин. По крайней мере, когда они заслуживают этого.
Люк: Всем нужно уважение, брат.
Джонни: Я знаю, что шутил.
Люк: Я знаю, что шутил.
Джонни: Ты хочешь пойти в клуб сегодня вечером? Я могу быть твоим помощником.
Джонни: Что это за пальцем в челюсть?
Люк: Зарегистрируйся на Tinder и начни охоту.
Джонни: Хахаха. xD
Джонни: Я понимаю, что ты не хочешь идти?
Люк: У меня сегодня есть планы. Другой раз, друг.
Джонни: Полегчай. Другой раз.

Джонни: Девушка в видео на вторник очень сексуальная.
Люк: <file_video>
Люк: На этот вторник?
Джонни: Она стоит того, чтобы нарушить правило "нет нута в ноябре".
Люк: Джонни, найди себе девушку. Влюбись в нее со всей душой и перестань говорить как подлец.


In [109]:
def yandex_preprocess(elem):
    # Here will be no history
    elem['question'], elem['answer'] = elem['question'].lower().strip(), elem['answer'].lower().strip()
    return {'texts': [format_dialogue('', elem['question'], elem['answer'])]}

# Throws error: proceed_dialogue isn't defined. That's because I use several processes to preprocess data
def diasum_preprocess(elem):
    dialogue = elem['dialogue']
    texts = proceed_dialogue(dialogue)
    return {'texts': texts}

# def diasum_preprocess(elem, sep=': '):
#     dialogue = elem['dialogue']
#     # remove empty lines
#     lines = list(filter(lambda x: x.strip(), dialogue.split('\n'))) 
#     # split by first ': ' to get speaker and text. Lower and strip text
#     splits = list(map(lambda x: x.lower().strip().split(sep, 1), lines)) 

#     res = [] # list of dialogues
#     history = '' # history of dialogues
#     prev_speaker, prev_text = splits[0] # first speaker and text

#     dialogue_dict = {prev_speaker: 1} # {speaker: number}
#     last = 2 # next number
#     for speaker, text in splits[1:]:
#         speaker, text = speaker.strip(), text.strip()
#         if speaker not in dialogue_dict:
#             dialogue_dict[speaker] = last
#             last += 1
#         res.append(format_dialogue(history, prev_text, text, dialogue_dict[prev_speaker], dialogue_dict[speaker]))
#         history += f'{dialogue_dict[prev_speaker]}: {prev_text}\t'
#         prev_speaker, prev_text = speaker, text
#     return res


In [110]:
n = np.random.randint(0, len(diasum))
pprint(diasum_preprocess(diasum[n]))

{'texts': ['history: \n'
           '1: добрый день. я получил ваши контактные данные от моей коллеги '
           'анны.\t2: здравствуйте, том, рады слышать от вас. анна сказала, '
           'что вы свяжетесь со мной. чем я могу вам помочь?',
           'history: 1: добрый день. я получил ваши контактные данные от моей '
           'коллеги анны.\t\n'
           '2: здравствуйте, том, рады слышать от вас. анна сказала, что вы '
           'свяжетесь со мной. чем я могу вам помочь?\t1: на самом деле, есть '
           'две вещи. во-первых, моя жена и я ищем квартиру. в то же время мы '
           'хотим продать нашу старую квартиру.',
           'history: 1: добрый день. я получил ваши контактные данные от моей '
           'коллеги анны.\t2: здравствуйте, том, рады слышать от вас. анна '
           'сказала, что вы свяжетесь со мной. чем я могу вам помочь?\t\n'
           '1: на самом деле, есть две вещи. во-первых, моя жена и я ищем '
           'квартиру. в то же время мы хотим про

In [111]:
pprint(yandex_preprocess(yandex[0]))

{'texts': ['history: \n'
           '1: как войти в роблокс, если не работает верификация, пишет '
           'слишком медленно?\t2: никак но можно попробовать найти сайты '
           'которые за тебя пройдут капчу за тебя попробуй это а так не '
           'вариант ну или зайди с другого браузера если с пк и пробуй опять']}


In [112]:
# We do need batches for speedup but only when tokenizing
diasum_texts = diasum.map(diasum_preprocess, batched=False, remove_columns=diasum.column_names)
yandex_texts = yandex.map(yandex_preprocess, batched=False, remove_columns=yandex.column_names)

In [113]:
yandex_texts[4]

{'texts': ['history: \n1: почему у ноутбука очень часто включается вентилятор?\t2: если у вас открыто 20 вкладок это особо не нагружает ноутбук, просто забивается оперативная память. открытое видео нагружает видео процессор. часто включается вентилятор понятие очень растяжимое тк зависит от множества факторов, какое устройство, на сколько часто итп. в некоторых ноутбуках вентилятор работает на постоянной основе просто на разных оборотах. всё зависит от того на сколько горячий чипсет на материнской плате. современные ноутбуки многие вообще без вентиляторов. на сколько я понимаю у вас на самом деле никакой проблемы нет, а максимум у вас забилась система охлаждения из-за этого затруднён отвод тепла и её нужно просто почистить и возможно заменить термопасту и термопрокладки.\nнадеюсь ответ был полезен. буду благодарен за оценку ответа и если остались вопросы пишите в комментариях с удовольствием отвечу. подписывайтесь на наши ответы и узнаете много полезного.']}

In [119]:
n = 37
print('Before')
print(diasum[n]['dialogue'])
print('After')
pprint(diasum_texts[n])

Before
Пегги: Вы можете порекомендовать зубного врача? У меня болит зуб:/
Шейн: Доктор Смит определенно, в клинике Центра.
Meg: Мне жаль о боли в зубах, Пегги:(
Meg: Я слышал, что он хороший, он поможет тебе.
Peggy: Спасибо, ребята! :*
After
{'texts': ['history: \n'
           '1: вы можете порекомендовать зубного врача? у меня болит зуб:/\t2: '
           'доктор смит определенно, в клинике центра.',
           'history: 1: вы можете порекомендовать зубного врача? у меня болит '
           'зуб:/\t\n'
           '2: доктор смит определенно, в клинике центра.\t3: мне жаль о боли '
           'в зубах, пегги:(',
           'history: 1: вы можете порекомендовать зубного врача? у меня болит '
           'зуб:/\t2: доктор смит определенно, в клинике центра.\t\n'
           '3: мне жаль о боли в зубах, пегги:(\t3: я слышал, что он хороший, '
           'он поможет тебе.',
           'history: 1: вы можете порекомендовать зубного врача? у меня болит '
           'зуб:/\t2: доктор смит опреде

In [121]:
texts = concatenate_datasets([diasum_texts, yandex_texts])

In [125]:
def explode_texts(elem):
    return {'text': sum(elem['texts'], [])}

texts = texts.map(explode_texts, batched=True, remove_columns=texts.column_names, num_proc=4)

In [127]:
!pip install transformers

     ---------------------------------------- 0.0/6.8 MB ? eta -:--:--
      --------------------------------------- 0.1/6.8 MB 1.7 MB/s eta 0:00:04
     - -------------------------------------- 0.2/6.8 MB 2.4 MB/s eta 0:00:03
     -- ------------------------------------- 0.5/6.8 MB 3.3 MB/s eta 0:00:02
     --- ------------------------------------ 0.6/6.8 MB 3.7 MB/s eta 0:00:02
     --- ------------------------------------ 0.6/6.8 MB 3.7 MB/s eta 0:00:02
     ---- ----------------------------------- 0.7/6.8 MB 2.5 MB/s eta 0:00:03
     ----- ---------------------------------- 1.0/6.8 MB 2.9 MB/s eta 0:00:02
     ------ --------------------------------- 1.1/6.8 MB 3.2 MB/s eta 0:00:02
     -------- ------------------------------- 1.4/6.8 MB 3.3 MB/s eta 0:00:02
     ---------- ----------------------------- 1.7/6.8 MB 3.6 MB/s eta 0:00:02
     ---------- ----------------------------- 1.8/6.8 MB 3.7 MB/s eta 0:00:02
     ----------- ---------------------------- 1.9/6.8 MB 3.3 MB/s eta 0

In [153]:
from transformers import AutoTokenizer
checkpoint = "AlexWortega/instruct_rugptMedium"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, pad_token='<|pad|>')
n = 10
special_tokens = {'additional_special_tokens': [f'{i}: ' for i in range(1, n)] + ['history: ']}
tokenizer.add_special_tokens(special_tokens)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


10

##### Don't forget to add special tokens to the model 
aka resize embeddings: model.resize_token_embeddings(len(tokenizer)) 

In [149]:
n = np.random.randint(0, len(texts))
inputs = tokenizer(texts[n]['text'])
# tokenizer.convert_ids_to_tokens(inputs["input_ids"])
pprint(tokenizer.decode(inputs['input_ids'][:10]))
pprint(tokenizer.convert_ids_to_tokens(inputs['input_ids'][:10]))

'history: \n1: как создать статью на яндекс.'
['history: ',
 'Ċ',
 '1: ',
 'ÐºÐ°Ðº',
 'ĠÑģÐ¾Ð·Ð´Ð°ÑĤÑĮ',
 'ĠÑģÑĤÐ°ÑĤÑĮÑİ',
 'ĠÐ½Ð°',
 'ĠÑıÐ½',
 'Ð´ÐµÐºÑģ',
 '.']


In [157]:
def encode_text(elem):
    inputs = tokenizer(elem['text'], padding='max_length', truncation=True, max_length=512, return_tensors='pt')
    return inputs

encoded_texts = texts.map(encode_text, batched=True, remove_columns=texts.column_names)

In [160]:
type(encoded_texts[0])

dict